In [1]:
import os

# Vérifie si le code est exécuté sur Google Colab
if 'COLAB_GPU' in os.environ:
    # Commandes à exécuter uniquement sur Google Colab
    !git clone https://github.com/GITHUB_ACCOUNT/tp-rag-student-version.git
    %cd tp-rag
    !pip install -r requirements.txt
else:
    # Commandes à exécuter si ce n'est pas sur Google Colab
    print("Pas sur Google Colab, ces commandes ne seront pas exécutées.")

Pas sur Google Colab, ces commandes ne seront pas exécutées.


In [2]:
# Installation des dépendances nécessaires (avec versions compatibles)
!pip install -q --upgrade langchain langchain-community langchain-core langchain-text-splitters langchain-huggingface chromadb sentence-transformers pypdf

In [8]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device utilisé : {device}")
if device == "cuda":
    print(f"GPU détecté : {torch.cuda.get_device_name(0)}")
    print(f"Mémoire GPU disponible : {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

Device utilisé : cuda
GPU détecté : NVIDIA GeForce RTX 3060 Ti
Mémoire GPU disponible : 8.59 GB


# TP - Retrieval Augmented Generation

## Étape 1 : Indexation des documents

## Exercice 1 : Indexation

In [3]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from pathlib import Path
import os

data_root = Path('data') if 'COLAB_GPU' not in os.environ else Path('tp-rag-student-version') / 'data'

text_exts = {'.txt', '.tex', '.bib'}
pdf_ext = '.pdf'

documents = []
data_files = []

if data_root.exists():
    for p in sorted(data_root.rglob('*')):
        if p.is_file() and p.suffix.lower() in text_exts.union({pdf_ext}):
            data_files.append(str(p))
else:
    print(f"Le chemin de données {data_root} n'existe pas")

for fp in data_files:
    p = Path(fp)
    if p.suffix.lower() in text_exts:
        try:
            loader = TextLoader(str(p), encoding='utf-8')
            docs = loader.load()
            documents.extend(docs)
        except Exception as e:
            print(f"Erreur en chargeant {p}: {e}")
    elif p.suffix.lower() == pdf_ext:
        try:
            pdf_loader = PyPDFLoader(str(p))
            docs = pdf_loader.load()
            documents.extend(docs)
        except Exception as e:
            print(f"Erreur en chargeant {p}: {e}")

print(f"Nombre total de fichiers trouvés : {len(data_files)}")
print(f"Nombre total de documents chargés : {len(documents)}")


C:\Users\lilia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Nombre total de fichiers trouvés : 65
Nombre total de documents chargés : 600


In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(documents)
print(f"Nombre de chunks créés : {len(splits)}")

Nombre de chunks créés : 9804


In [5]:
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

print("Modèle d'embeddings chargé : intfloat/multilingual-e5-base")

C:\Users\lilia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lilia\.cache\huggingface\hub\models--intfloat--multilingual-e5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Modèle d'embeddings chargé : intfloat/multilingual-e5-base


In [9]:
persist_directory = "chroma_db"

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=persist_directory
)

print(f"Nombre de documents indexés : {vectorstore._collection.count()}")

Nombre de documents indexés : 11489


## Exercice 2 : Interrogation

In [11]:
def search_documents(query, k=5):
    """
    Interroge la base vectorielle et retourne les documents les plus pertinents.
    
    Args:
        query (str): La requête de recherche
        k (int): Le nombre de documents à retourner (défaut: 5)
    
    Returns:
        list: Liste de tuples (document, score) avec les documents pertinents et leurs scores
    """
    results = vectorstore.similarity_search_with_score(query, k=k)
    
    return results


def display_search_results(results):
    """
    Affiche les résultats de recherche de manière formatée.
    
    Args:
        results (list): Liste de tuples (document, score)
    """
    print(f"\n{'='*80}")
    print(f"Nombre de résultats : {len(results)}")
    print(f"{'='*80}\n")
    
    for i, (doc, score) in enumerate(results, 1):
        print(f"Résultat {i} - Score de similarité : {score:.4f}")
        print(f"Source : {doc.metadata.get('source', 'Unknown')}")
        print(f"Contenu : {doc.page_content[:300]}...")
        print(f"{'-'*80}\n")

In [13]:
query1 = "Explain DrivingGPT."
results1 = search_documents(query1, k=3)
display_search_results(results1)


Nombre de résultats : 3

Résultat 1 - Score de similarité : 0.2646
Source : data\autres_articles\2412.18607v1.pdf
Contenu : marks demonstrates the effectiveness of the proposed Driv-
ingGPT on action-conditioned video generation and end-
to-end planning. DrivingGPT clearly shows the viability of
unified learning of world modeling and planning with a sin-
gle model, setting a stepping stone for the future exploration
of d...
--------------------------------------------------------------------------------

Résultat 2 - Score de similarité : 0.2835
Source : data\autres_articles\2412.18607v1.pdf
Contenu : DrivingGPT: Unifying Driving World Modeling and Planning with Multi-modal
Autoregressive Transformers
Supplementary Material
A. Refining Video Generation with SVD De-
coder.
Since independently decoding each frame-token to pixel
space leads to temporally inconsistent video outputs, we
further employ...
--------------------------------------------------------------------------------

Résu

In [14]:
query2 = "Comment mesurer la pression artérielle ?"
results2 = search_documents(query2, k=3)
display_search_results(results2)


Nombre de résultats : 3

Résultat 1 - Score de similarité : 0.2764
Source : data\patents\APPARATUS FOR THE MEASUREMENT OF ATRIAL PRESSURE.txt
Contenu : EP	0615422	B1	2000-01-19	en	TITLE	1	APPARATUS FOR THE MEASUREMENT OF ATRIAL PRESSURE...
--------------------------------------------------------------------------------

Résultat 2 - Score de similarité : 0.2764
Source : data/patents/APPARATUS FOR THE MEASUREMENT OF ATRIAL PRESSURE.txt
Contenu : EP	0615422	B1	2000-01-19	en	TITLE	1	APPARATUS FOR THE MEASUREMENT OF ATRIAL PRESSURE...
--------------------------------------------------------------------------------

Résultat 3 - Score de similarité : 0.2945
Source : data\arxiv\A_Survey_on_Blood_Pressure_Measurement_Technologies_Addressing__Potential_Sources_of_Bias.pdf
Contenu : still remain popular in clinical settings.
2) Oscillometric: This method is currently the most popular
technology for automated BP measurement devices [6]. Using
a pressure sensor, it measures the pulsatile BP in t

# Étape 2 - RAG

## Exercice 3 : Prompt Template

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

rag_prompt_template = """Tu es un assistant intelligent et utile qui répond aux questions en te basant sur le contexte fourni.

Utilise UNIQUEMENT les informations du contexte ci-dessous pour répondre à la question. Si le contexte ne contient pas l'information nécessaire pour répondre, dis-le clairement.

Contexte :
{context}

Question : {question}

Instructions :
- Réponds de manière claire et concise
- Cite les sources si pertinent
- Si l'information n'est pas dans le contexte, indique-le clairement
- Utilise un ton professionnel mais accessible

Réponse :"""

prompt = ChatPromptTemplate.from_template(rag_prompt_template)

print(f"\nVariables du template : {prompt.input_variables}")


Variables du template : ['context', 'question']


## Exercice 4 : Chaîne RAG

In [21]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = ChatOllama(
    model="qwen3:8b",
    temperature=0,
)

In [22]:
def format_docs(docs):
    """Formatte les documents récupérés pour le contexte."""
    return "\n\n".join(doc.page_content for doc in docs)


def rag_chain(question, vectorstore, prompt_template, llm, k=3):
    """
    Chaîne RAG complète : récupération + génération de réponse.
    
    Args:
        question (str): La question de l'utilisateur
        vectorstore: Le vector store contenant les documents
        prompt_template: Le template de prompt à utiliser
        llm: Le modèle de langage
        k (int): Nombre de documents à récupérer
    
    Returns:
        str: La réponse générée
    """
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})
    
    chain = (
        {
            "context": retriever | format_docs,
            "question": RunnablePassthrough()
        }
        | prompt_template
        | llm
        | StrOutputParser()
    )
    
    response = chain.invoke(question)
    
    return response

In [24]:
question1 = "Qu'est-ce que le Multi-Agent Reinforcement Learning et quelles sont ses applications ?"

print("Question :", question1)
print("\n" + "="*80)
response1 = rag_chain(question1, vectorstore, prompt, llm, k=3)
print("Réponse :", response1)
print("="*80)

Question : Qu'est-ce que le Multi-Agent Reinforcement Learning et quelles sont ses applications ?

Réponse : Le **Multi-Agent Reinforcement Learning (MARL)** est une technique d'intelligence artificielle (IA) qui permet à plusieurs agents (robots, programmes, etc.) d'apprendre à collaborer ou compétiter en s'adaptant aux interactions mutuelles dans un environnement. Il vise à optimiser les stratégies collectives en tenant compte des comportements des autres agents.  

**Applications** :  
Le contexte mentionne que le MARL est utilisé dans des scénarios d'apprentissage coopératif, notamment pour des tâches nécessitant la collaboration entre agents (ex. : gestion de ressources, contrôle de systèmes distribués). Des méthodes comme **LOLA** (Learning with Opponent-Learning Awareness) illustrent comment les agents peuvent développer des politiques coopératives en tenant compte des actions des autres. Cependant, le contexte ne fournit pas d'exemples concrets d'applications industrielles ou s

In [26]:
question2 = "Comment mesurer la pression artérielle et quelles sont les sources d'erreur potentielles ?"

print("Question :", question2)
print("\n" + "="*80)
response2 = rag_chain(question2, vectorstore, prompt, llm, k=3)
print("Réponse :", response2)
print("="*80)

Question : Comment mesurer la pression artérielle et quelles sont les sources d'erreur potentielles ?

Réponse : Pour mesurer la pression artérielle, les méthodes courantes incluent :  
- **Mesure en cabinet** (avec sphygmomanomètre manuel ou dispositif automatique).  
- **Mesure hors cabinet** (monitorage ambulatoire ou mesure 24 heures).  
- **Auto-mesure** (dispositifs portables utilisés à domicile).  

**Sources d'erreur potentielles** :  
1. **Technique inadéquate** (position incorrecte, inflation insuffisante du ballonnet).  
2. **Calibration incorrecte** des dispositifs (défaillances ou manque de maintenance).  
3. **Facteurs patient** (mouvement, anxiété, caféine, ou prise de médicaments).  
4. **Environnement** (bruit, température, ou interférences électromagnétiques).  
5. **Erreurs de lecture** (interprétation erronée des données par les patients).  

Ces points sont abordés dans les guides de l'European Society of Hypertension ([43]) et dans une revue systématique sur les e

## Exercice 5 : Mémoire

In [30]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

contextualize_q_system_prompt = """Étant donné un historique de conversation et la dernière question de l'utilisateur 
qui pourrait faire référence au contexte de l'historique de conversation, formulez une question autonome 
qui peut être comprise sans l'historique de conversation. Ne répondez PAS à la question, 
reformulez-la simplement si nécessaire, sinon retournez-la telle quelle."""

contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

qa_system_prompt = """Tu es un assistant intelligent et utile qui répond aux questions en te basant sur le contexte fourni.

Utilise UNIQUEMENT les informations du contexte ci-dessous pour répondre à la question. 
Si le contexte ne contient pas l'information nécessaire pour répondre, dis-le clairement.

Contexte :
{context}

Instructions :
- Réponds de manière claire et concise
- Cite les sources si pertinent
- Si l'information n'est pas dans le contexte, indique-le clairement
- Utilise un ton professionnel mais accessible
"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

In [31]:
def create_conversational_rag_chain(vectorstore, llm):
    """
    Crée une chaîne RAG avec gestion de l'historique de conversation.
    
    Args:
        vectorstore: Le vector store contenant les documents
        llm: Le modèle de langage
    
    Returns:
        Une chaîne RAG conversationnelle
    """
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )
    
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    
    return rag_chain

conversational_rag = create_conversational_rag_chain(vectorstore, llm)

In [32]:
chat_history = []

question1 = "Qu'est-ce que le Multi-Agent Reinforcement Learning ?"
response1 = conversational_rag.invoke({
    "input": question1,
    "chat_history": chat_history
})

print("Question 1 :", question1)
print("Réponse 1 :", response1["answer"])
print("\n" + "="*80 + "\n")

chat_history.extend([
    HumanMessage(content=question1),
    AIMessage(content=response1["answer"])
])

question2 = "Quelles sont ses principales applications ?"
response2 = conversational_rag.invoke({
    "input": question2,
    "chat_history": chat_history
})

print("Question 2 :", question2)
print("Réponse 2 :", response2["answer"])
print("\n" + "="*80 + "\n")

chat_history.extend([
    HumanMessage(content=question2),
    AIMessage(content=response2["answer"])
])

question3 = "Parle-moi des méthodes de mesure de la pression artérielle"
response3 = conversational_rag.invoke({
    "input": question3,
    "chat_history": chat_history
})

print("Question 3 :", question3)
print("Réponse 3 :", response3["answer"])

Question 1 : Qu'est-ce que le Multi-Agent Reinforcement Learning ?
Réponse 1 : Le **Multi-Agent Reinforcement Learning (MARL)** est un domaine de la science des données et de l'intelligence artificielle qui étudie comment plusieurs agents (robots, programmes, etc.) apprennent à prendre des décisions collaboratives ou compétitives dans un environnement partagé. Les agents ajustent leurs stratégies en fonction des interactions avec leur environnement et les autres agents, en maximisant un critère de récompense collectif ou individuel.

### Points clés du contexte :
1. **Défis et méthodes** :  
   - Des approches comme **LOLA** (Learning with Opponent-Learning Awareness) intègrent l'impact des politiques d'un agent sur les mises à jour des paramètres des autres, favorisant la coopération (Foerster et al., 2018).  
   - Des attaques adversariales sont utilisées pour renforcer la robustesse des politiques face aux perturbations (ex. : perturbations des observations).  

2. **Applications** 

## Exercice 6 : Nouveaux Outils

In [34]:
from langchain_classic.chains.summarize import load_summarize_chain
from langchain_core.prompts import PromptTemplate

def get_document_by_title(title_keyword, data_files, documents):
    """
    Recherche un document par mot-clé dans le titre.
    
    Args:
        title_keyword (str): Mot-clé à rechercher dans le nom du fichier
        data_files (list): Liste des chemins de fichiers
        documents (list): Liste des documents chargés
    
    Returns:
        list: Documents correspondants
    """
    matching_docs = []
    
    for i, file_path in enumerate(data_files):
        if title_keyword.lower() in file_path.lower():
            doc_chunks = [doc for doc in documents if doc.metadata.get('source', '') == file_path]
            matching_docs.extend(doc_chunks)
    
    return matching_docs


def summarize_document(title_keyword, llm, data_files, documents):
    """
    Résume un document complet identifié par un mot-clé.
    
    Args:
        title_keyword (str): Mot-clé pour identifier le document
        llm: Le modèle de langage
        data_files (list): Liste des fichiers de données
        documents (list): Liste des documents chargés
    
    Returns:
        str: Résumé du document
    """
    doc_chunks = get_document_by_title(title_keyword, data_files, documents)
    
    if not doc_chunks:
        return f"Aucun document trouvé contenant '{title_keyword}'"
    
    source = doc_chunks[0].metadata.get('source', 'Unknown')
    print(f"Document trouvé : {source}")
    print(f"Nombre de chunks : {len(doc_chunks)}")
    
    summary_prompt = PromptTemplate(
        template="""Écris un résumé concis et structuré du texte suivant.
        
        Texte :
        {text}
        
        Résumé structuré :""",
        input_variables=["text"]
    )
    
    chain = load_summarize_chain(
        llm,
        chain_type="map_reduce",
        map_prompt=summary_prompt,
        combine_prompt=summary_prompt,
        verbose=False
    )
    summary = chain.run(doc_chunks)
    
    return summary

In [35]:
summary = summarize_document(
    "Multi-Agent Reinforcement Learning", 
    llm, 
    data_files, 
    documents
)

print("\n" + "="*80)
print("RÉSUMÉ DU DOCUMENT")
print("="*80)
print(summary)
print("="*80)

Document trouvé : data\latex\Multi-Agent Reinforcement Learning, Methods, Applications, Visionary Prospects, and Challenges.bib
Nombre de chunks : 2


C:\Users\lilia\AppData\Local\Temp\ipykernel_29556\4156399873.py:65: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  summary = chain.run(doc_chunks)
C:\Users\lilia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\language_models\base.py:328: UserWarning: Using fallback GPT-2 tokenizer for token counting. Token counts may be inaccurate for non-GPT-2 models. For accurate counts, use a model-specific method if available.
  return len(self.get_token_ids(text))
C:\Users\lilia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lilia\.cache\hugg


RÉSUMÉ DU DOCUMENT
**Résumé structuré :**  

### **1. Applications médicales et scientifiques**  
- **Diagnostic médical** : Utilisation de RL pour améliorer la précision des diagnostics (cancer du sein, interprétation des symptômes) et optimiser les traitements via des systèmes d’assistance.  
- **Recherche scientifique** : Applications en physique (turbulence, contrôle de plasmas tokamak) et gestion de systèmes complexes (fusion, énergétique).  

### **2. Algorithmes et méthodes RL**  
- **Techniques clés** : A3C (optimisation asynchrone), MAAC (multi-agents avec attention), et frameworks en temps réel pour gérer des données dynamiques.  
- **Systèmes multi-agents** : Coordination d’agents dans des environnements complexes (ex. contrôle de plasmas, gestion de trafic).  

### **3. Défis des systèmes HCPS (humain-centrés)**  
- **Non-stationnarité** : Les comportements humains perturbent l’environnement, rendant les systèmes instables.  
- **Diversité humaine** : Variations culturelle

In [36]:
summary2 = summarize_document(
    "Blood_Pressure", 
    llm, 
    data_files, 
    documents
)

print("\n" + "="*80)
print("RÉSUMÉ DU DOCUMENT")
print("="*80)
print(summary2)
print("="*80)

Document trouvé : data\arxiv\A_Survey_on_Blood_Pressure_Measurement_Technologies_Addressing__Potential_Sources_of_Bias.pdf
Nombre de chunks : 22

RÉSUMÉ DU DOCUMENT
**Résumé structuré :**  

### **1. Contexte et importance**  
- La mesure de la pression artérielle (PAP) est essentielle pour évaluer le risque cardiovasculaire, mais les dispositifs actuels (manuels, automatisés) présentent des biais techniques, physiologiques et environnementaux (âge, sexe, température, bruit), pouvant mener à des erreurs de diagnostic et de traitement.  

### **2. Méthodes de mesure et standards**  
- **Invasive** : Cathéter + transducteur (précision élevée, utilisée en soins intensifs).  
- **Non invasive** :  
  - **Cuff-based** : Korotkoff (manuelle) ou oscillométrique (automatisée).  
  - **Cuff-less** : Tonométrie, PPG (en recherche, plus confortables).  
- **Normes** : ISO 81060-2 (2018) et FDA (510(k)) pour valider la précision (erreur <5 mmHg, écart type <8 mmHg).  

### **3. Facteurs influençan

# Étape 3 - IHM

## Exercice 7 : IHM

In [37]:
!pip install -q gradio

In [ ]:
import gradio as gr
from pathlib import Path
from langchain_classic.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain


def advanced_chatbot(message, history, k_docs, show_sources):
    """
    Traite une question avec le système RAG et retourne une réponse formatée.
    
    Args:
        message: Question de l'utilisateur (string)
        history: Historique de conversation Gradio (liste de dictionnaires)
        k_docs: Nombre de documents à récupérer
        show_sources: Si True, ajoute les sources à la réponse
    
    Returns:
        Réponse formatée avec sources optionnelles (string)
    """
    # Convertir l'historique Gradio (format dictionnaire) en format LangChain
    chat_history = []
    for msg_dict in history:
        if isinstance(msg_dict, dict):
            role = msg_dict.get("role", "")
            content = msg_dict.get("content", "")
            if role == "user":
                chat_history.append(HumanMessage(content=content))
            elif role == "assistant":
                chat_history.append(AIMessage(content=content))
    
    # Créer une chaîne RAG avec les paramètres personnalisés
    retriever = vectorstore.as_retriever(search_kwargs={"k": k_docs})
    
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    
    # Générer la réponse
    response = rag_chain.invoke({
        "input": str(message),  # S'assurer que c'est une chaîne
        "chat_history": chat_history
    })
    
    answer = response["answer"]
    
    # Ajouter les sources si demandé
    if show_sources and "context" in response:
        sources = {Path(doc.metadata.get("source", "Unknown")).name 
                   for doc in response["context"]}
        
        if sources:
            answer += "\n\n" + "─" * 80 + "\n"
            answer += "**Sources utilisées :**\n"
            for i, source in enumerate(sorted(sources), 1):
                answer += f"  {i}. {source}\n"
    
    return answer


def search_interface(query, k):
    """
    Recherche sémantique dans la base vectorielle.
    
    Args:
        query: Requête de recherche
        k: Nombre de résultats à retourner
    
    Returns:
        Résultats formatés en Markdown
    """
    if not query.strip():
        return "Veuillez entrer une requête de recherche."
    
    results = search_documents(query, k=k)
    
    output = f"# Résultats de recherche\n\n"
    output += f"**{len(results)} documents trouvés pour :** *\"{query}\"*\n\n"
    output += "═" * 80 + "\n\n"
    
    for i, (doc, score) in enumerate(results, 1):
        source_name = Path(doc.metadata.get('source', 'Unknown')).name
        
        output += f"## Résultat {i}\n"
        output += f"**Score de similarité :** {score:.4f}  \n"
        output += f"**Source :** `{source_name}`\n\n"
        output += f"**Extrait :**\n```\n{doc.page_content[:400]}...\n```\n\n"
        output += "─" * 80 + "\n\n"
    
    return output


def summarize_interface(keyword):
    """
    Génère un résumé d'un document complet.
    
    Args:
        keyword: Mot-clé pour identifier le document
    
    Returns:
        Résumé du document ou message d'erreur
    """
    if not keyword.strip():
        return "Veuillez entrer un mot-clé pour rechercher un document."
    
    try:
        summary = summarize_document(keyword, llm, data_files, documents)
        return f"# Résumé du document\n\n{summary}"
    except Exception as e:
        return f"Erreur lors de la génération du résumé : {str(e)}"


def get_available_docs():
    """
    Génère une liste formatée des documents disponibles.
    
    Returns:
        Liste des documents en Markdown
    """
    doc_list = "# Documents disponibles\n\n"
    
    # Grouper par catégorie
    categories = {
        'arxiv': ('Articles arXiv', [f for f in data_files if 'arxiv' in f]),
        'autres': ('Autres articles', [f for f in data_files if 'autres_articles' in f]),
        'patents': ('Brevets', [f for f in data_files if 'patents' in f])
    }
    
    for title, docs in categories.values():
        if docs:
            doc_list += f"## {title}\n\n"
            for doc in docs[:10]:
                doc_list += f"- `{Path(doc).name}`\n"
            if len(docs) > 10:
                doc_list += f"- *... et {len(docs) - 10} autres*\n"
            doc_list += "\n"
    
    doc_list += f"**Total : {len(data_files)} documents indexés**"
    
    return doc_list


# ============================================================================
# INTERFACE GRADIO
# ============================================================================

with gr.Blocks(theme=gr.themes.Soft(), title="Système RAG Complet") as demo:
    
    gr.Markdown("""
    # Système RAG - Assistant Documentaire Intelligent
    
    Explorez une base de connaissances scientifique avec un assistant IA avancé.
    """)
    
    with gr.Tabs():
        
        # ─────────────────────────────────────────────────────────────────
        # ONGLET 1 : CHAT
        # ─────────────────────────────────────────────────────────────────
        with gr.TabItem("Chat", id="chat"):
            with gr.Row():
                with gr.Column(scale=3):
                    chatbot = gr.Chatbot(
                        label="Conversation",
                        height=450
                    )
                    
                    with gr.Row():
                        msg = gr.Textbox(
                            label="Votre question",
                            placeholder="Posez votre question sur les documents...",
                            lines=2,
                            scale=4,
                            show_label=False
                        )
                        submit = gr.Button("Envoyer", scale=1, variant="primary")
                    
                    clear = gr.Button("Nouvelle conversation", size="sm")
                
                with gr.Column(scale=1):
                    gr.Markdown("### Paramètres")
                    
                    k_docs = gr.Slider(
                        minimum=1,
                        maximum=10,
                        value=3,
                        step=1,
                        label="Nombre de documents",
                        info="Documents à consulter par requête"
                    )
                    
                    show_sources = gr.Checkbox(
                        label="Afficher les sources",
                        value=True,
                        info="Cite les documents utilisés"
                    )
                    
                    gr.Markdown("### Exemples")
                    gr.Examples(
                        examples=[
                            "Qu'est-ce que le Multi-Agent Reinforcement Learning ?",
                            "Comment mesurer la pression artérielle ?",
                            "Parle-moi de la sécurité des smart grids",
                            "Qu'est-ce que l'inflation targeting ?",
                        ],
                        inputs=msg
                    )
            
            # Gestion des événements du chat
            def respond(message, chat_history, k, sources):
                # Générer la réponse
                bot_message = advanced_chatbot(message, chat_history, k, sources)
                # Ajouter à l'historique au format dictionnaire
                chat_history.append({"role": "user", "content": message})
                chat_history.append({"role": "assistant", "content": bot_message})
                return "", chat_history
            
            msg.submit(respond, [msg, chatbot, k_docs, show_sources], [msg, chatbot], queue=False)
            submit.click(respond, [msg, chatbot, k_docs, show_sources], [msg, chatbot], queue=False)
            clear.click(lambda: None, None, chatbot, queue=False)
        
        # ─────────────────────────────────────────────────────────────────
        # ONGLET 2 : RECHERCHE
        # ─────────────────────────────────────────────────────────────────
        with gr.TabItem("Recherche", id="search"):
            gr.Markdown("### Recherche sémantique dans la base vectorielle")
            
            with gr.Row():
                search_query = gr.Textbox(
                    label="Requête de recherche",
                    placeholder="Entrez votre recherche (ex: quantum computing)...",
                    lines=2,
                    scale=3
                )
                k_search = gr.Slider(
                    minimum=1,
                    maximum=10,
                    value=5,
                    step=1,
                    label="Nombre de résultats",
                    scale=1
                )
            
            search_btn = gr.Button("Rechercher", variant="primary", size="lg")
            search_output = gr.Markdown(label="Résultats", value="*Aucune recherche effectuée*")
            
            gr.Examples(
                examples=[
                    "reinforcement learning applications",
                    "blood pressure measurement techniques",
                    "smart grid security threats",
                    "quantum state learning complexity",
                ],
                inputs=search_query
            )
            
            search_btn.click(
                search_interface,
                inputs=[search_query, k_search],
                outputs=search_output
            )
        
        # ─────────────────────────────────────────────────────────────────
        # ONGLET 3 : RÉSUMÉ
        # ─────────────────────────────────────────────────────────────────
        with gr.TabItem("Résumé", id="summary"):
            gr.Markdown("### Génération de résumés de documents complets")
            
            with gr.Row():
                with gr.Column(scale=1):
                    doc_keyword = gr.Textbox(
                        label="Mot-clé du document",
                        placeholder="Ex: Multi-Agent, Blood_Pressure, Smart_Grid...",
                        lines=1
                    )
                    
                    summarize_btn = gr.Button(
                        "Générer le résumé",
                        variant="primary",
                        size="lg"
                    )
                    
                    summary_output = gr.Markdown(
                        label="Résumé généré",
                        value="*Aucun résumé généré*"
                    )
                
                with gr.Column(scale=1):
                    docs_list = gr.Markdown(
                        label="Documents disponibles",
                        value=get_available_docs()
                    )
                    
                    refresh_docs = gr.Button("Rafraîchir", size="sm")
            
            summarize_btn.click(
                summarize_interface,
                inputs=doc_keyword,
                outputs=summary_output
            )
            
            refresh_docs.click(
                get_available_docs,
                outputs=docs_list
            )
    
    gr.Markdown("""
    ---
    ### Guide d'utilisation rapide
    
    | Onglet | Description | Utilisation |
    |--------|-------------|-------------|
    | **Chat** | Conversez avec l'assistant | Posez des questions, l'IA répond avec le contexte des documents |
    | **Recherche** | Recherche sémantique | Trouvez les passages les plus pertinents dans la base |
    | **Résumé** | Résumés de documents | Générez des synthèses structurées de documents entiers |
    """)

demo.launch(
    share=True,
    debug=True,
    show_error=True
)


C:\Users\lilia\AppData\Local\Temp\ipykernel_29556\162278108.py:148: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), title="Système RAG Complet") as demo:


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://f58e516c43c0de97c8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f58e516c43c0de97c8.gradio.live


# Etape 4 : Evaluation

## Exercice 8 : Evaluation

VOIR [L6](./multi_agent_data/notebooks/L6/L6_évolution_Monitoring,_au_RAG_SQL,_à_l'Optimisation.ipynb) + Ajouter clé grok cellule 2